In [74]:
import tensorflow as tf
import numpy as np
import random
import cv2
path = '/home/mmtao/test/lanenet-lane-detection/data/tusimple_test_image/05250538_0313.MP4/00000.jpg'
label_path = '/home/mmtao/data/CULane/laneseg_label_w16/driver_23_30frame/05151640_0419.MP4/00000.png'
image = cv2.cvtColor(cv2.imread(path,-1), cv2.COLOR_BGR2RGB)
label_in = cv2.imread(label_path,cv2.IMREAD_COLOR)
ret, label = cv2.threshold(label_in,0,255,cv2.THRESH_BINARY)


#cv2.imshow("dd",image)
#cv2.waitKey(2000)
#cv2.destroyAllWindows()

crop_height = 576
crop_width = 1600
        
if (crop_width <= image.shape[1]) and (crop_height <= image.shape[0]):
    x = random.randint(0, image.shape[1]-crop_width)
    y = random.randint(0, image.shape[0]-crop_height)
    
    if len(label.shape) == 3:
        image_result = image[y:y+crop_height, x:x+crop_width, :]
        label_result = label[y:y+crop_height, x:x+crop_width, :]
    else:
        image_result = image[y:y+crop_height, x:x+crop_width, :]
        label_result = label[y:y+crop_height, x:x+crop_width]
else:
    Exception('Crop shape (%d, %d) exceeds image dimensions (%d, %d)!' % (crop_height, crop_width, image.shape[0], image.shape[1]))
    
cv2.imshow("result1",image_result)
cv2.imshow("result2",label_result)
cv2.waitKey(30000)
cv2.destroyAllWindows()


In [68]:
path = '/home/mmtao/test/lanenet-lane-detection/data/tusimple_test_image/05250538_0313.MP4/00000.jpg'
image = cv2.cvtColor(cv2.imread(path,-1), cv2.COLOR_BGR2RGB)

In [75]:
a = 10
b = 91
print(b/a)


9.1


In [104]:
import tensorflow as tf
import numpy as np
import os

def sigmoid(x):
    return 1.0/(1+np.exp(-x))

y = np.array([[1,0,0],[0,1,0],[0,0,1],[1,1,0],[0,1,0]])

logits = np.array([[12,3,2],[3,10,1],[1,2,5],[4,6.5,1.2],[3,6,1]])
y_pred = sigmoid(logits)
E1 = -y*np.log(y_pred)-(1-y)*np.log(1-y_pred)
print(E1)
print(y.shape)
print(np.shape(logits)[0])
print(E1.shape)
print('-------------')

with tf.device('/cpu:0'):
    sess =tf.Session()
    y = np.array(y).astype(np.float64)
    #sess.run(tf.global_variables_initializer())
    E2 = sess.run(tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=logits))
    print(E2)

[[6.14419348e-06 3.04858735e+00 2.12692801e+00]
 [3.04858735e+00 4.53988992e-05 1.31326169e+00]
 [1.31326169e+00 2.12692801e+00 6.71534849e-03]
 [1.81499279e-02 1.50231016e-03 1.46328247e+00]
 [3.04858735e+00 2.47568514e-03 1.31326169e+00]]
(5, 3)
5
(5, 3)
-------------


InternalError: CUDA runtime implicit initialization on GPU:3 failed. Status: out of memory

In [1]:
import tensorflow as tf
import numpy as np

def softmax(x):
    sum_raw = np.sum(np.exp(x),axis=-1)
    x1 = np.ones(np.shape(x))
    for i in range(np.shape(x)[0]):
        print(x[i])
        x1[i] = np.exp(x[i])/sum_raw[i]
    return x1

y = np.array([[1,0,0],[0,1,0],[0,0,1],[1,0,0],[0,1,0]])

logits =np.array([[12,3,2],[3,10,1],[1,2,5],[4,6.5,1.2],[3,6,1]])

y_pred =softmax(logits)
E1 = -np.sum(y*np.log(y_pred),-1)
print(E1)
sess = tf.Session()
y = np.array(y).astype(np.float64)
E2 = sess.run(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
print(E2)


[12.  3.  2.]
[ 3. 10.  1.]
[1. 2. 5.]
[4.  6.5 1.2]
[3. 6. 1.]
[1.68795487e-04 1.03475622e-03 6.58839038e-02 2.58349207e+00
 5.49852354e-02]


InternalError: CUDA runtime implicit initialization on GPU:3 failed. Status: out of memory

In [117]:
import tensorflow as tf
import numpy as np
def softmax(x):
    """
    Compute the softmax function for each row of the input x.

    Arguments:
    x -- A N dimensional vector or M x N dimensional numpy matrix.

    Return:
    x -- You are allowed to modify x in-place
    """
    orig_shape = x.shape

    if len(x.shape) > 1:
        # Matrix
        exp_minmax = lambda x: np.exp(x - np.max(x))
        denom = lambda x: 1.0 / np.sum(x)
        x = np.apply_along_axis(exp_minmax, 1, x)
        denominator = np.apply_along_axis(denom, 1, x)

        if len(denominator.shape) == 1:
            denominator = denominator.reshape((denominator.shape[0], 1))

        x = x * denominator
    else:
        # Vector
        x_max = np.max(x)
        x = x - x_max
        numerator = np.exp(x)
        denominator = 1.0 / np.sum(numerator)
        x = numerator.dot(denominator)

    assert x.shape == orig_shape
    return x

y = np.array([[1,0,0],[0,1,0],[0,0,1],[1,0,0],[0,1,0]])

logits =np.array([[12,3,2],[3,10,1],[1,2,5],[4,6.5,1.2],[3,6,1]])

y_pred =softmax(logits)
E1 = -np.sum(y*np.log(y_pred),-1)
print(E1)
print(y_pred)


[1.68795487e-04 1.03475622e-03 6.58839038e-02 2.58349207e+00
 5.49852354e-02]
[[9.99831219e-01 1.23388975e-04 4.53922671e-05]
 [9.10938878e-04 9.98965779e-01 1.23282171e-04]
 [1.71478255e-02 4.66126226e-02 9.36239552e-01]
 [7.55098575e-02 9.19898383e-01 4.59175917e-03]
 [4.71234165e-02 9.46499123e-01 6.37746092e-03]]


In [7]:
import tensorflow as tf
import numpy as np
import random
import cv2

path = '/home/mmtao/test/Semantic-Segmentation-Suite/CamVid/train/0001TP_006690.png'

#image = cv2.cvtColor(cv2.imread(path,-1), cv2.COLOR_BGR2RGB)
image = cv2.imread(path, cv2.IMREAD_COLOR)

cv2.imshow("dd",image)
cv2.waitKey(9000)
cv2.destroyAllWindows()

In [15]:
import tensorflow as tf
import numpy as np

epoch_start_i = 0
num_epochs = 30
num_iters = 10000
learning_rate_init = 0.025

for epoch in range(epoch_start_i, num_epochs):
    iter_index = 0
    for i in range(num_iters):
        # st=time.time()
        b = (1.0-(i+num_iters*iter_index)/((num_epochs-epoch_start_i)*num_iters))**(0.9)
        learning_rate_update = learning_rate_init*b
        #print(learning_rate_update)
        #print((1.0-(i+num_iters*iter_index)/((num_epochs-epoch_start_i)*num_iters))**(0.9))
        if i%1000 ==0:
            print('.................')
            print(b)
            print(learning_rate_update)
        #learning_rate_init = learning_rate_update
    iter_index+=1
    learning_rate_init = learning_rate_update

.................
1.0
0.025
.................
0.9969994993878173
0.024924987484695432
.................
0.9939979950939289
0.024849949877348225
.................
0.9909954834128342
0.024774887085320857
.................
0.9879919606128271
0.02469979901532068
.................
0.9849874229357218
0.02462468557339305
.................
0.9819818665965753
0.024549546664914382
.................
0.9789752877834037
0.024474382194585094
.................
0.9759676826568975
0.024399192066422437
.................
0.9729590473501305
0.024323976183753263
.................
1.0
0.0242488097038868
.................
0.9969994993878173
0.024176051135525584
.................
0.9939979950939289
0.024103268229077688
.................
0.9909954834128342
0.02403046089468912
.................
0.9879919606128271
0.023957629041870467
.................
0.9849874229357218
0.023884772579490184
.................
0.9819818665965753
0.023811891415767908
.................
0.9789752877834037
0.02373898545826757
.......

In [18]:
import tensorflow as tf
temp4=tf.reshape(tf.range(0,20)+tf.constant(1,shape=[20]),[2,2,5])
with tf.Session() as sess:
    print(sess.run(temp4))

[[[ 1  2  3  4  5]
  [ 6  7  8  9 10]]

 [[11 12 13 14 15]
  [16 17 18 19 20]]]
